#**Import Data And Library**


In [35]:
# from google.colab import drive
# drive.mount('/content/drive')

In [36]:
!pip install pyvi

In [37]:
import tensorflow as tf
import pandas as pd
import numpy as np
from string import digits
from collections import Counter
from pyvi import ViTokenizer
from gensim.models.word2vec import Word2Vec
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
%matplotlib inline

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, MaxPooling1D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Reshape, Embedding , Flatten, Dropout, Dense, BatchNormalization, concatenate, Bidirectional
from tensorflow.keras.layers import LeakyReLU, Multiply, Activation, RepeatVector, Permute
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ReduceLROnPlateau
from transformers import TFBertModel, BertTokenizer

In [39]:
import os

In [40]:
# !cp -r "/content/drive/MyDrive/NLP_Modern" /content

In [41]:
# %cd "/content/NLP_Modern"

In [42]:
data_train = pd.read_csv("E:/Code/BKU/HK3/NLP/data/vlsp_sentiment_train.csv", sep='\t')
data_train.columns =['Class', 'Data']
data_test = pd.read_csv("E:/Code/BKU/HK3/NLP/data/vlsp_sentiment_test.csv", sep='\t')
data_test.columns =['Class', 'Data']

In [43]:
print(data_train.shape)
print(data_test.shape)

(5100, 2)
(1050, 2)



#**Preprocessing**


In [44]:
labels = data_train.iloc[:, 0].values
reviews = data_train.iloc[:, 1].values

In [45]:
encoded_labels = []

for label in labels:
    if label == -1:
        encoded_labels.append([1,0,0])
    elif label == 0:
        encoded_labels.append([0,1,0])
    else:
        encoded_labels.append([0,0,1])

encoded_labels = np.array(encoded_labels)

In [46]:
reviews_processed = []
unlabeled_processed = []
for review in reviews:
    review_cool_one = ''.join([char for char in review if char not in digits])
    reviews_processed.append(review_cool_one)

In [47]:
#Use PyVi for Vietnamese word tokenizer
word_reviews = []
all_words = []
for review in reviews_processed:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews.append(review.split())


In [48]:
EMBEDDING_DIM = 400 # how big is each word vector
MAX_VOCAB_SIZE = 10000 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 300 # max number of words in a comment to use

In [49]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(word_reviews)
sequences_train = tokenizer.texts_to_sequences(word_reviews)
word_index = tokenizer.word_index

In [50]:
data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
labels = encoded_labels

In [51]:
print('Shape of X train and X validation tensor:',data.shape)
print('Shape of label train and validation tensor:', labels.shape)

Shape of X train and X validation tensor: (5100, 300)
Shape of label train and validation tensor: (5100, 3)


#**Prepare data for test**

In [52]:
labels_test = data_test.iloc[:, 0].values
reviews_test = data_test.iloc[:, 1].values

In [53]:
encoded_labels_test = []

for label_test in labels_test:
    if label_test == -1:
        encoded_labels_test.append([1,0,0])
    elif label_test == 0:
        encoded_labels_test.append([0,1,0])
    else:
        encoded_labels_test.append([0,0,1])

encoded_labels_test = np.array(encoded_labels_test)

In [54]:
reviews_processed_test = []
unlabeled_processed_test = []
for review_test in reviews_test:
    review_cool_one = ''.join([char for char in review_test if char not in digits])
    reviews_processed_test.append(review_cool_one)

In [55]:
#Use PyVi for Vietnamese word tokenizer
word_reviews_test = []
all_words = []
for review_test in reviews_processed_test:
    review_test = ViTokenizer.tokenize(review_test.lower())
    word_reviews_test.append(review_test.split())

In [56]:
sequences_test = tokenizer.texts_to_sequences(word_reviews_test)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
labels_test = encoded_labels_test

In [57]:
print('Shape of X train and X validation tensor:',data_test.shape)
print('Shape of label train and validation tensor:', labels_test.shape)

Shape of X train and X validation tensor: (1050, 300)
Shape of label train and validation tensor: (1050, 3)


Word Vector

In [58]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('vi-model-CBOW.bin', binary=True)


vocabulary_size=min(len(word_index)+1,MAX_VOCAB_SIZE)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=MAX_VOCAB_SIZE:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [59]:
vocab_size = 10000
sequence_length = data.shape[1]
num_filters = 128 
filter_sizes = [3, 4, 5]
drop = 0.5
inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)

**CNN**

In [351]:
# Improved CNN Architecture
conv_0 = Conv1D(num_filters, filter_sizes[0], activation='relu', kernel_regularizer=regularizers.l2(0.001))(embedding)
conv_0 = BatchNormalization()(conv_0)
conv_0 = Dropout(0.3)(conv_0)  # Increased Dropout in Conv layers

conv_1 = Conv1D(num_filters, filter_sizes[1], activation='relu', kernel_regularizer=regularizers.l2(0.001))(embedding)
conv_1 = BatchNormalization()(conv_1)
conv_1 = Dropout(0.3)(conv_1)  # Increased Dropout in Conv layers

conv_2 = Conv1D(num_filters, filter_sizes[2], activation='relu', kernel_regularizer=regularizers.l2(0.001))(embedding)
conv_2 = BatchNormalization()(conv_2)
conv_2 = Dropout(0.3)(conv_2)  # Increased Dropout in Conv layers

maxpool_0 = GlobalMaxPooling1D()(conv_0)
avgpool_0 = GlobalAveragePooling1D()(conv_0)

maxpool_1 = GlobalMaxPooling1D()(conv_1)
avgpool_1 = GlobalAveragePooling1D()(conv_1)

maxpool_2 = GlobalMaxPooling1D()(conv_2)
avgpool_2 = GlobalAveragePooling1D()(conv_2)

# Added another conv layer for more complex feature learning
conv_3 = Conv1D(num_filters, filter_sizes[0], activation='relu', kernel_regularizer=regularizers.l2(0.001))(embedding)
conv_3 = BatchNormalization()(conv_3)
conv_3 = Dropout(0.3)(conv_3)

maxpool_3 = GlobalMaxPooling1D()(conv_3)
avgpool_3 = GlobalAveragePooling1D()(conv_3)

merged_tensor = concatenate([maxpool_0, avgpool_0, maxpool_1, avgpool_1, maxpool_2, avgpool_2, maxpool_3, avgpool_3])
flatten = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(merged_tensor)
dropout = Dropout(0.5)(flatten)
output = Dense(units=3, activation='softmax', kernel_regularizer=regularizers.l2(0.001))(dropout)

modelCNN = Model(inputs, output)

# Smaller learning rate for potentially better results
adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
modelCNN.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

modelCNN.summary()

#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=6, verbose=1)
callbacks_list = [early_stopping]

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "functional_37"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_35      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_34        │ (None, 300, 400)  │  3,167,600 │ input_layer_35[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_97 (Conv1D)  │ (None, 298, 128)  │    153,728 │ embedding_34[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_98 (Conv1D)  │ (None, 297, 128)  │    204,928 │ embedding_34[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_99 (Conv1D)  │ (None, 296, 128)  │    256,128 │ embedding_34[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_100 (Conv1D) │ (None, 298, 128)  │    153,728 │ embedding_34[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 298, 128)  │        512 │ conv1d_97[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 297, 128)  │        512 │ conv1d_98[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 296, 128)  │        512 │ conv1d_99[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 298, 128)  │        512 │ conv1d_100[0][0]  │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_166         │ (None, 298, 128)  │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_167         │ (None, 297, 128)  │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_168         │ (None, 296, 128)  │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_169         │ (None, 298, 128)  │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ dropout_166[0][0] │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ dropout_166[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ dropout_167[0][0] │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ dropout_167[0][0] │
│ (GlobalAveragePool… │                   │            │                 

 Total params: 4,201,331 (16.03 MB)

 Trainable params: 4,200,307 (16.02 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [352]:
modelCNN.fit(data, labels, validation_split=0.2,
            epochs=16, batch_size=128, callbacks=callbacks_list, shuffle=True)

Epoch 1/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 12s 317ms/step - accuracy: 0.4151 - loss: 10.6451 - val_accuracy: 0.0549 - val_loss: 2.4981
Epoch 2/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 304ms/step - accuracy: 0.5382 - loss: 2.0855 - val_accuracy: 9.8039e-04 - val_loss: 2.5891
Epoch 3/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 305ms/step - accuracy: 0.6026 - loss: 1.9422 - val_accuracy: 9.8039e-04 - val_loss: 2.5736
Epoch 4/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 304ms/step - accuracy: 0.6382 - loss: 1.8328 - val_accuracy: 0.0020 - val_loss: 2.4462
Epoch 5/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 306ms/step - accuracy: 0.6764 - loss: 1.7134 - val_accuracy: 0.0510 - val_loss: 2.2288
Epoch 6/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 597ms/step - accuracy: 0.7058 - loss: 1.6382 - val_accuracy: 0.0206 - val_loss: 2.3705
Epoch 7/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 844ms/step - accuracy: 0.7209 - loss: 1.5715 - val_accuracy: 0.0304 - val_loss: 2.4636
Epoch 8/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 844ms/step - accuracy: 0.7711 - loss: 1.4678 -

In [353]:
# Evaluate the model
loss, accuracy = modelCNN.evaluate(data, labels, verbose=0)

# Print the accuracy
print(f'Test Accuracy: {accuracy * 100:.2f}%')
print(f'Test Loss: {loss * 100:.2f}%')

Test Accuracy: 72.10%
Test Loss: 156.89%


#**LSTM**

In [60]:
# Define input layer
inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)

reshape = Reshape((sequence_length,EMBEDDING_DIM))(embedding)

# LSTM layers
lstm = LSTM(num_filters, return_sequences=True, kernel_regularizer=regularizers.l2(0.01))(reshape)
lstm = BatchNormalization()(lstm)
lstm = LSTM(num_filters, return_sequences=False, kernel_regularizer=regularizers.l2(0.01))(lstm)
lstm = BatchNormalization()(lstm)

# Fully connected layers
dropout = Dropout(drop)(lstm)
output = Dense(units=3, activation='softmax', kernel_regularizer=regularizers.l2(0.01))(dropout)  # Ensure this matches the number of classes

# Create model
modelLSTM = Model(inputs, output)

# Optimizer with learning rate schedule
adam = Adam(learning_rate=0.001)

modelLSTM.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

callbacks_list = [early_stopping, reduce_lr]

modelLSTM.summary()



Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 300, 400)       │     3,167,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_3 (Reshape)             │ (None, 300, 400)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 300, 128)       │       270,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 300, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,571,443 (13.62 MB)

 Trainable params: 3,570,931 (13.62 MB)

 Non-trainable params: 512 (2.00 KB)

In [61]:
modelLSTM.fit(data, labels, validation_split=0.2,
              epochs=16, batch_size=128, callbacks=callbacks_list, shuffle=True)

Epoch 1/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 554ms/step - accuracy: 0.4030 - loss: 8.0687 - val_accuracy: 0.0039 - val_loss: 6.9871 - learning_rate: 0.0010
Epoch 2/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 17s 528ms/step - accuracy: 0.5761 - loss: 6.4792 - val_accuracy: 0.0029 - val_loss: 6.1926 - learning_rate: 0.0010
Epoch 3/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 17s 527ms/step - accuracy: 0.6776 - loss: 5.3017 - val_accuracy: 0.0539 - val_loss: 5.3240 - learning_rate: 0.0010
Epoch 4/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 17s 528ms/step - accuracy: 0.7632 - loss: 4.3168 - val_accuracy: 0.1569 - val_loss: 4.5814 - learning_rate: 0.0010
Epoch 5/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 17s 527ms/step - accuracy: 0.8220 - loss: 3.5290 - val_accuracy: 0.1725 - val_loss: 3.9812 - learning_rate: 0.0010
Epoch 6/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 17s 527ms/step - accuracy: 0.8690 - loss: 2.8864 - val_accuracy: 0.0275 - val_loss: 4.1489 - learning_rate: 0.0010
Epoch 7/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 17s 525ms/step - accuracy: 0.8938 - loss: 2.

In [62]:
# Evaluate the model
loss, accuracy = modelLSTM.evaluate(data, labels, verbose=0)

# Print the accuracy
print(f'Test Accuracy: {accuracy * 100:.2f}%')
print(f'Test Loss: {loss * 100:.2f}%')

Test Accuracy: 72.02%
Test Loss: 253.31%



#**Combining CNN and LSTM**

In [323]:

# Define your input layer
inputs = Input(shape=(sequence_length,))

# Embedding layer
embedding = Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, input_length=sequence_length)(inputs)

# CNN layers to extract features
conv_layers = []
for filter_size in filter_sizes:
    conv = Conv1D(num_filters, filter_size, activation='relu', kernel_regularizer=regularizers.l2(0.01))(embedding)
    conv = BatchNormalization()(conv)
    conv = Dropout(0.3)(conv)  # Increased dropout in convolutional layers
    conv = MaxPooling1D(sequence_length - filter_size + 1, strides=1)(conv)
    conv_layers.append(conv)

# Concatenate convolutional layers
merged_tensor = concatenate(conv_layers, axis=1)

# Reshape for LSTM input
reshape = Reshape((1, len(filter_sizes) * num_filters))(merged_tensor)

# Bidirectional LSTM layers
lstm = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)))(reshape)
lstm = BatchNormalization()(lstm)
lstm = Dropout(0.3)(lstm)  # Increased dropout in LSTM layers

# Flatten the tensor
flatten = Flatten()(lstm)

# Dense layer with more units
dense = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(flatten)
dropout_dense = Dropout(drop)(dense)

# Output layer
output = Dense(units=3, activation='softmax', kernel_regularizer=regularizers.l2(0.01))(dropout_dense)

# Create the model
modelCNN_LSTM = Model(inputs, output)

# Compile the model with a smaller learning rate
adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
modelCNN_LSTM.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Model summary
modelCNN_LSTM.summary()

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=6, verbose=1)
callbacks_list = [early_stopping]


Model: "functional_36"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_34      │ (None, 300)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_33        │ (None, 300, 400)  │  4,000,000 │ input_layer_34[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_94 (Conv1D)  │ (None, 298, 128)  │    153,728 │ embedding_33[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_95 (Conv1D)  │ (None, 297, 128)  │    204,928 │ embedding_33[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_96 (Conv1D)  │ (None, 296, 128)  │    256,128 │ embedding_33[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 298, 128)  │        512 │ conv1d_94[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 297, 128)  │        512 │ conv1d_95[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 296, 128)  │        512 │ conv1d_96[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_161         │ (None, 298, 128)  │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_162         │ (None, 297, 128)  │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_163         │ (None, 296, 128)  │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_39    │ (None, 1, 128)    │          0 │ dropout_161[0][0] │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_40    │ (None, 1, 128)    │          0 │ dropout_162[0][0] │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_41    │ (None, 1, 128)    │          0 │ dropout_163[0][0] │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_58      │ (None, 3, 128)    │          0 │ max_pooling1d_39… │
│ (Concatenate)       │                   │            │ max_pooling1d_40… │
│                     │                   │            │ max_pooling1d_41… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_31          │ (None, 1, 384)    │          0 │ concatenate_58[0… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_67    │ (None, 1, 256)    │    525,312 │ reshape_31[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 5,209,219 (19.87 MB)

 Trainable params: 5,207,939 (19.87 MB)

 Non-trainable params: 1,280 (5.00 KB)

In [324]:
# Training the model (Assuming 'data' and 'labels' are defined)
modelCNN_LSTM.fit(data, labels, validation_split=0.2,
                  epochs=16, batch_size=128, callbacks=callbacks_list, shuffle=True)


Epoch 1/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 244ms/step - accuracy: 0.4099 - loss: 17.2481 - val_accuracy: 0.0000e+00 - val_loss: 13.5992
Epoch 2/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 231ms/step - accuracy: 0.6435 - loss: 12.0079 - val_accuracy: 0.0000e+00 - val_loss: 9.9725
Epoch 3/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 231ms/step - accuracy: 0.8310 - loss: 8.2494 - val_accuracy: 0.0000e+00 - val_loss: 7.6464
Epoch 4/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 230ms/step - accuracy: 0.9645 - loss: 5.6299 - val_accuracy: 0.0000e+00 - val_loss: 5.8801
Epoch 5/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 230ms/step - accuracy: 0.9711 - loss: 3.9731 - val_accuracy: 0.0000e+00 - val_loss: 4.7388
Epoch 6/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 230ms/step - accuracy: 0.9837 - loss: 2.8174 - val_accuracy: 0.0000e+00 - val_loss: 4.0061
Epoch 7/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 229ms/step - accuracy: 0.9835 - loss: 2.0274 - val_accuracy: 0.0000e+00 - val_loss: 3.5805
Epoch 8/16
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 230ms/step - accuracy: 0.9823 -

In [325]:
# Evaluate the model
loss, accuracy = modelCNN_LSTM.evaluate(data, labels, verbose=0)

# Print the accuracy
print(f'Test Accuracy: {accuracy * 100:.2f}%')
print(f'Test Loss: {loss * 100:.2f}%')

Test Accuracy: 77.43%
Test Loss: 101.27%


#**BERT-based approach**

In [66]:
print(inputs)

<KerasTensor shape=(None, 300), dtype=float32, sparse=False, name=keras_tensor_42>


In [70]:
# # Đọc dữ liệu
# data_train = pd.read_csv("E:/Code/BKU/HK3/NLP/data/vlsp_sentiment_train.csv", sep='\t')
# data_train.columns = ['Class', 'Data']
# data_test = pd.read_csv("E:/Code/BKU/HK3/NLP/data/vlsp_sentiment_test.csv", sep='\t')
# data_test.columns = ['Class', 'Data']

# # Xử lý nhãn (mã hóa nhãn thành one-hot encoding)
# y_train = to_categorical(data_train['Class'], num_classes=3)  # Giả sử có 3 lớp phân loại
# y_test = to_categorical(data_test['Class'], num_classes=3)

# # BERT Tokenizer
# bert_model_name = 'bert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(bert_model_name)

# # Tokenize dữ liệu
# train_encodings = tokenizer(list(data_train['Data']), truncation=True, padding=True, max_length=sequence_length, return_tensors='tf')
# test_encodings = tokenizer(list(data_test['Data']), truncation=True, padding=True, max_length=sequence_length, return_tensors='tf')

# # Định nghĩa input layer cho mô hình BERT
# input_layer = Input(shape=(sequence_length,), dtype=tf.int32)

# # Sử dụng mô hình BERT cho Sequence Classification
# bert = TFBertForSequenceClassification.from_pretrained(bert_model_name, num_labels=3)
# output = bert(input_layer)[0]  # Output từ lớp classification head

# # Thêm các lớp Dense và Dropout
# dense = Dense(256, activation='relu')(output)
# dropout = Dropout(0.3)(dense)

# # Lớp output với softmax activation cho phân loại cảm xúc
# final_output = Dense(3, activation='softmax')(dropout)

# # Tạo mô hình
# model = Model(inputs=input_layer, outputs=final_output)

# # Biên dịch mô hình
# adam = Adam(learning_rate=0.0001)
# model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# # Hiển thị tóm tắt mô hình
# model.summary()

# # Định nghĩa các callback
# early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

# # Huấn luyện mô hình
# model.fit(train_encodings['input_ids'], y_train, 
#           validation_data=(test_encodings['input_ids'], y_test),
#           epochs=10, batch_size=32, callbacks=[early_stopping])


In [71]:
# modelBERT.fit(data, labels, validation_split=0.2,
#               epochs=16, batch_size=128, callbacks=callbacks_list, shuffle=True)

In [72]:
# # Evaluate the model
# loss, accuracy = modelBERT.evaluate(data, labels, verbose=0)

# # Print the accuracy
# print(f'Test Accuracy: {accuracy * 100:.2f}%')
# print(f'Test Loss: {loss * 100:.2f}%')